In [ ]:
import numpy as np
import pandas as pd


import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
sn.set_style(style="whitegrid")

In [ ]:
df=pd.read_csv("/kaggle/input/covid-2020/1Abriltotalcases.csv",
               index_col=["date"],parse_dates=["date"])

In [ ]:
df

In [ ]:
cases_mexico=df.Mexico[df.Mexico>1]
cases_mexico

# Exploratory analysis

In [ ]:
def show_histplot_boxplot():

    plt.subplots(1,1,figsize=(10,7))
    sn.histplot(cases_mexico,kde=True,color="c")
    plt.show() 
    
    sn.boxplot(cases_mexico,color="c")
    plt.show()


In [ ]:
show_histplot_boxplot()

Detect outlires values.

# Feature enginiering

In [ ]:
lower = cases_mexico.quantile(0.25)
upper=cases_mexico.quantile(0.999)

We define a lower and upper interval in order to minimize outliers.

In [ ]:
def eliminate_out(lower,upper):
    
    return df.Mexico[(df.Mexico > lower) & (df.Mexico < upper)]

In [ ]:
cases_mexico_not_out=eliminate_out(lower,upper)

In [ ]:
sn.boxplot(cases_mexico_not_out)
plt.show()

In [ ]:
cases_mexico_not_out

In [ ]:
days=np.arange(15,32,1)
target=np.array(cases_mexico_not_out)

In [ ]:
plt.figure(figsize=(12,10))
plt.title("Covid 19 cases 2020 from Mexico")
plt.scatter(days,target)

plt.xlabel("Days")
plt.ylabel("Cases covid-19")
plt.show()

# Scipy

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def func(x,c,k):
    
    return c*np.exp(k*x)


We will use this equation to represent the data since they are exponential in nature.

In [ ]:
equation,covariance_matrix=curve_fit(func,days,target)

In [ ]:
y_pred=func(days,*equation)

In [ ]:
y_pred

In [ ]:
mexico_cases=pd.DataFrame({"Days": days, 
                           "Cases" : cases_mexico_not_out})

In [ ]:
mexico_cases["prediction"]= y_pred

In [ ]:
mexico_cases

The model have predictons very close respect to tryue values but failled with lower values.

Compare the results of the true values vs predicted values.

In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score

In [ ]:
print("MAE")

print(mean_absolute_error(mexico_cases.Cases,mexico_cases.prediction))

In [ ]:
print("R2 Score")


print(r2_score(mexico_cases.Cases,mexico_cases.prediction))

The model fits true values but has a MAE mode because it has a greater margin of error with the first values.

In [ ]:
def show_curve_fit(ax):

    
    return mexico_cases.plot.scatter(x="Days",y="prediction",ax=ax,color="c",figsize=(10,10))
    

In [ ]:
ax=mexico_cases.plot(x="Days",y="prediction",color="r")

plt.title("True values vs prediction values")
show_curve_fit(ax)
plt.show()